<a href="https://colab.research.google.com/github/anaya729/Twitter-sentiment-analysis/blob/main/Twitter_Sentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install kaggle

In [ ]:
#configure the path of kaggle.json file
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [5]:
#api to fetch dataset
!kaggle datasets download -d kazanova/sentiment140

 96% 78.0M/80.9M [00:00<00:00, 176MB/s]
100% 80.9M/80.9M [00:00<00:00, 181MB/s]


In [7]:
#extracting compressed dataset
from zipfile import ZipFile
dataset = '/content/sentiment140.zip'
with ZipFile(dataset,'r') as zip:
  zip.extractall()
print('The dataset is extracted')

The dataset is extracted


In [9]:
#libraries
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


In [10]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [11]:
#processing
twitter_data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv',encoding='latin-1')

In [12]:
#checking rows and columns
twitter_data.shape

(1599999, 6)

In [13]:
twitter_data.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [14]:
#Naming columns
column_names = ['target','id','date','flag','user','tweet']
twitter_data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv',names = column_names,encoding='latin-1')

In [15]:
twitter_data.head()

,target,id,date,flag,user,tweet
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [16]:
#counting missing values
twitter_data.isnull().sum()

target    0
id        0
date      0
flag      0
user      0
tweet     0
dtype: int64

In [17]:
# checking distribution of tweets
twitter_data['target'].value_counts()

target
0    800000
4    800000
Name: count, dtype: int64

In [21]:
# converting 4 to 1
twitter_data.replace({'target':{4:1}}, inplace = True)

In [22]:
twitter_data['target'].value_counts()

target
0    800000
1    800000
Name: count, dtype: int64

In [23]:
# Stemming
port_stem = PorterStemmer()

In [24]:
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]',' ',content)
    sremmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

In [25]:
twitter_data['stemmed_content'] = twitter_data['tweet'].apply(stemming)

In [26]:
print(twitter_data['stemmed_content'])

0          switchfoot http twitpic com zl awww bummer you...
1          upset updat facebook text might cri result sch...
2          kenichan i dive mani time ball manag save the ...
3                            whole bodi feel itchi like fire
4                            nationwideclass behav mad i see
                                 ...                        
1599995                 just woke have school best feel ever
1599996    thewdb com veri cool hear old walt interview h...
1599997                     are readi mojo makeov ask detail
1599998    happi th birthday boo alll time tupac amaru sh...
1599999    happi charitytuesday thenspcc sparkschar speak...
Name: stemmed_content, Length: 1600000, dtype: object


In [28]:
print(twitter_data['target'])

0          0
1          0
2          0
3          0
4          0
          ..
1599995    1
1599996    1
1599997    1
1599998    1
1599999    1
Name: target, Length: 1600000, dtype: int64


In [29]:
from tkinter.constants import X
# separating data and label
x = twitter_data['stemmed_content'].values
y = twitter_data['target'].values

In [30]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=2)

In [31]:
print(x.shape, x_train.shape, x_test.shape)

(1600000,) (1280000,) (320000,)


In [32]:
# converting textual to numerical
vectorizer = TfidfVectorizer()
x_train = vectorizer.fit_transform(x_train)
x_test = vectorizer.transform(x_test)

In [33]:
print(x_train)

  (0, 129962)	0.6592576617698773
  (0, 31234)	0.7519171067329923
  (1, 437931)	0.8042773910733999
  (1, 241540)	0.5942540519072339
  (2, 411106)	0.18464287904169105
  (2, 278622)	0.4853139770765464
  (2, 145734)	0.16444046079173225
  (2, 140241)	0.22372378686138533
  (2, 416239)	0.2970321685021026
  (2, 93642)	0.16612263205577082
  (2, 128294)	0.28170394757881806
  (2, 42840)	0.22940430929074537
  (2, 237087)	0.6367700241935804
  (3, 286700)	0.3682809811482607
  (3, 8560)	0.37668420201866026
  (3, 445770)	0.2188627839232472
  (3, 179108)	0.4788299006944713
  (3, 308928)	0.41292329558269175
  (3, 201943)	0.524207488360723
  (4, 227675)	0.2958646150787124
  (4, 415838)	0.2474136975967658
  (4, 442058)	0.32896947543200367
  (4, 315365)	0.2958290290894173
  (4, 302756)	0.32329111062510013
  (4, 59754)	0.28951346889086854
  :	:
  (1279996, 286201)	0.19295841733643207
  (1279996, 445265)	0.1645728738109807
  (1279997, 336374)	0.6258956159372502
  (1279997, 127893)	0.4659041039141294
  (12799

In [34]:
print(x_test)

  (0, 445831)	0.1686037286425815
  (0, 428035)	0.3348580570741247
  (0, 414690)	0.18641633598338916
  (0, 404205)	0.18336369255515522
  (0, 387026)	0.15901770353744457
  (0, 371843)	0.2058863475317293
  (0, 334343)	0.1803548674924732
  (0, 317505)	0.17947600748216608
  (0, 313601)	0.3305146358581725
  (0, 298829)	0.13286829532553548
  (0, 280279)	0.40487394201504584
  (0, 272369)	0.30857448615427746
  (0, 220175)	0.12415166239928044
  (0, 177723)	0.1267567069713972
  (0, 149580)	0.09891495120108752
  (0, 53071)	0.37905610839107184
  (0, 5622)	0.2960922269114722
  (1, 154453)	0.7963195549499829
  (1, 30476)	0.6048761579069399
  (2, 384716)	0.5275602488060214
  (2, 344216)	0.45616935735592273
  (2, 306763)	0.4614836588658659
  (2, 43580)	0.434390421943326
  (2, 39328)	0.3345556683317226
  (3, 239081)	0.432753389073631
  :	:
  (319996, 419163)	0.3618037526316817
  (319996, 408760)	0.4646424847685505
  (319996, 385839)	0.41302888896674317
  (319996, 358037)	0.20613515112091785
  (319996, 3

In [41]:
# Training the model
model = LogisticRegression(max_iter=1000)

In [42]:
model.fit(x_train, y_train)

LogisticRegression(max_iter=1000)

In [39]:
# Model evaluation & accuracy
x_train_prediction = model.predict(x_train)
training_data_accuracy = accuracy_score(x_train_prediction, y_train)

In [40]:
print('Accuracy score of the training data : ', training_data_accuracy)

Accuracy score of the training data :  0.81136953125


In [43]:
x_test_prediction = model.predict(x_test)
test_data_accuracy = accuracy_score(x_test_prediction, y_test)

In [44]:
print('Accuracy score of the test data : ', test_data_accuracy)

Accuracy score of the test data :  0.781771875


In [45]:
# Saving model
import pickle

In [46]:
filename = 'trained_model.sav'
pickle.dump(model, open(filename, 'wb'))

In [47]:
#Loading the saved modal
loaded_model = pickle.load(open('trained_model.sav', 'rb'))

In [51]:
x_new = x_test[200]
prediction = loaded_model.predict(x_new)
print(prediction)

if (prediction[0]==0):
  print('Negative')
else:
  print('Positive')

[0]
Negative
